<a href="https://colab.research.google.com/github/ipejun-ai/m5-accuracy/blob/master/check_imporatance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# General imports
import numpy as np
import pandas as pd
import os, sys, gc, time, warnings, pickle, psutil, random

# custom imports
from multiprocessing import Pool        # Multiprocess Runs

warnings.filterwarnings('ignore')

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
DIRPATH="/content/gdrive/My Drive/kaggle/"
#DIRPATH="C:/Users/peiju/Documents/Study/kaggle/m5-forecasting-accuracy/"

In [4]:
!pip install Boruta

In [0]:
from boruta import BorutaPy
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from boruta import BorutaPy
import lightgbm as lgb
import xgboost as xgb
from sklearn.utils import check_random_state

In [0]:
#########LGM wrapper


class BorutaPyForLGB(BorutaPy):
    def __init__(self, estimator, n_estimators=1000, perc=100, alpha=0.05,
                 two_step=True, max_iter=100, random_state=None, verbose=0):
        super().__init__(estimator, n_estimators, perc, alpha,
                         two_step, max_iter, random_state, verbose)
        if random_state is None:
            self.random_state_input = np.random.randint(0, 2**64-1)
        elif isinstance(random_state, int):
            self.random_state_input = random_state
        else:
            raise TypeError('random_state must be int or None')

    def _get_tree_num(self, n_feat):
        depth = self.estimator.get_params()['max_depth']
        if (depth == None) or (depth <= 0):
            depth = 10
        f_repr = 100
        multi = ((n_feat * 2) / (np.sqrt(n_feat * 2) * depth))
        n_estimators = int(multi * f_repr)
        return n_estimators

    def _fit(self, X, y):
        # check input params
        self._check_params(X, y)
        self.random_state = check_random_state(self.random_state)
        # setup variables for Boruta
        n_sample, n_feat = X.shape
        _iter = 1
        # holds the decision about each feature:
        # 0  - default state = tentative in original code
        # 1  - accepted in original code
        # -1 - rejected in original code
        dec_reg = np.zeros(n_feat, dtype=np.int)
        # counts how many times a given feature was more important than
        # the best of the shadow features
        hit_reg = np.zeros(n_feat, dtype=np.int)
        # these record the history of the iterations
        imp_history = np.zeros(n_feat, dtype=np.float)
        sha_max_history = []

        # set n_estimators
        if self.n_estimators != 'auto':
            self.estimator.set_params(n_estimators=self.n_estimators)

        # main feature selection loop
        while np.any(dec_reg == 0) and _iter < self.max_iter:
            # find optimal number of trees and depth
            if self.n_estimators == 'auto':
                # number of features that aren't rejected
                not_rejected = np.where(dec_reg >= 0)[0].shape[0]
                n_tree = self._get_tree_num(not_rejected)
                self.estimator.set_params(n_estimators=n_tree)

            # make sure we start with a new tree in each iteration
            self.estimator.set_params(random_state=self.random_state_input)

            # add shadow attributes, shuffle them and train estimator, get imps
            cur_imp = self._add_shadows_get_imps(X, y, dec_reg)

            # get the threshold of shadow importances we will use for rejection
            imp_sha_max = np.percentile(cur_imp[1], self.perc)

            # record importance history
            sha_max_history.append(imp_sha_max)
            imp_history = np.vstack((imp_history, cur_imp[0]))

            # register which feature is more imp than the max of shadows
            hit_reg = self._assign_hits(hit_reg, cur_imp, imp_sha_max)

            # based on hit_reg we check if a feature is doing better than
            # expected by chance
            dec_reg = self._do_tests(dec_reg, hit_reg, _iter)

            # print out confirmed features
            if self.verbose > 0 and _iter < self.max_iter:
                self._print_results(dec_reg, _iter, 0)
            if _iter < self.max_iter:
                _iter += 1

        # we automatically apply R package's rough fix for tentative ones
        confirmed = np.where(dec_reg == 1)[0]
        tentative = np.where(dec_reg == 0)[0]
        # ignore the first row of zeros
        tentative_median = np.median(imp_history[1:, tentative], axis=0)
        # which tentative to keep
        tentative_confirmed = np.where(tentative_median
                                       > np.median(sha_max_history))[0]
        tentative = tentative[tentative_confirmed]

        # basic result variables
        self.n_features_ = confirmed.shape[0]
        self.support_ = np.zeros(n_feat, dtype=np.bool)
        self.support_[confirmed] = 1
        self.support_weak_ = np.zeros(n_feat, dtype=np.bool)
        self.support_weak_[tentative] = 1

        # ranking, confirmed variables are rank 1
        self.ranking_ = np.ones(n_feat, dtype=np.int)
        # tentative variables are rank 2
        self.ranking_[tentative] = 2
        # selected = confirmed and tentative
        selected = np.hstack((confirmed, tentative))
        # all rejected features are sorted by importance history
        not_selected = np.setdiff1d(np.arange(n_feat), selected)
        # large importance values should rank higher = lower ranks -> *(-1)
        imp_history_rejected = imp_history[1:, not_selected] * -1

        # update rank for not_selected features
        if not_selected.shape[0] > 0:
                # calculate ranks in each iteration, then median of ranks across feats
                iter_ranks = self._nanrankdata(imp_history_rejected, axis=1)
                rank_medians = np.nanmedian(iter_ranks, axis=0)
                ranks = self._nanrankdata(rank_medians, axis=0)

                # set smallest rank to 3 if there are tentative feats
                if tentative.shape[0] > 0:
                    ranks = ranks - np.min(ranks) + 3
                else:
                    # and 2 otherwise
                    ranks = ranks - np.min(ranks) + 2
                self.ranking_[not_selected] = ranks
        else:
            # all are selected, thus we set feature supports to True
            self.support_ = np.ones(n_feat, dtype=np.bool)

        # notify user
        if self.verbose > 0:
            self._print_results(dec_reg, _iter, 1)
        return self

In [0]:
########################### Helpers
#################################################################################
## Seeder
# :seed to make all processes deterministic     # type: int
def seed_everything(seed=0):
    random.seed(seed)
    np.random.seed(seed)

    
## Multiprocess Runs
def df_parallelize_run(func, t_split):
    num_cores = np.min([N_CORES,len(t_split)])
    pool = Pool(num_cores)
    df = pd.concat(pool.map(func, t_split), axis=1)
    pool.close()
    pool.join()
    return df

In [0]:
########################### Helper to load data by store ID
#################################################################################
# Read data
def get_data_by_store(store):
    
    # Read and contact basic feature
    df = pd.concat([pd.read_pickle(BASE),
                    pd.read_pickle(PRICE).iloc[:,2:],
                    pd.read_pickle(CALENDAR).iloc[:,2:]],
                    axis=1)
    
    # Leave only relevant store
    df = df[df['store_id']==store]

    # With memory limits we have to read 
    # lags and mean encoding features
    # separately and drop items that we don't need.
    # As our Features Grids are aligned 
    # we can use index to keep only necessary rows
    # Alignment is good for us as concat uses less memory than merge.
    df2 = pd.read_pickle(MEAN_ENC)[mean_features]
    df2 = df2[df2.index.isin(df.index)]
    
    df3 = pd.read_pickle(LAGS).iloc[:,3:]
    df3 = df3[df3.index.isin(df.index)]


    df4 = pd.read_pickle(SNAP).iloc[:,2:]
    df4 = df4[df4.index.isin(df.index)]

    df5 = pd.read_pickle(EVENT_TYPE).iloc[:,2:]
    df5 = df5[df5.index.isin(df.index)]

    df6 = pd.read_pickle(EVENT_VALUE).iloc[:,2]
    df6 = df6[df6.index.isin(df.index)]

    df7 = pd.read_pickle(LAGS_364).iloc[:,2:]
    df7 = df7[df7.index.isin(df.index)]

    df8 = pd.read_pickle(HOLIDAY).iloc[:,2:]
    df8 = df8[df8.index.isin(df.index)]

    #df5 = pd.read_pickle(EVENT_TYPE).iloc[:,2:]
    #df5 = df5[df5.index.isin(df.index)]
    
    df = pd.concat([df, df2], axis=1)
    del df2 # to not reach memory limit 
    
    df = pd.concat([df, df3], axis=1)
    del df3 # to not reach memory limit 

    df = pd.concat([df, df4], axis=1)
    del df4 # to not reach memory limit 

    df = pd.concat([df, df5], axis=1)
    del df5 # to not reach memory limit 

    df = pd.concat([df, df6], axis=1)
    del df6 # to not reach memory limit 

    df = pd.concat([df, df7], axis=1)
    del df7 # to not reach memory limit 

    df = pd.concat([df, df8], axis=1)
    del df8 # to not reach memory limit 

   

    
    # Create features list
    features = [col for col in list(df) if col not in remove_features]
    df = df[['id','d',TARGET]+features]
    
    # Skipping first n rows
    df = df[df['d']>=START_TRAIN].reset_index(drop=True)
    
    return df, features

# Recombine Test set after training
def get_base_test():
    base_test = pd.DataFrame()

    for store_id in STORES_IDS:
        temp_df = pd.read_pickle('test_'+store_id+'.pkl')
        temp_df['store_id'] = store_id
        base_test = pd.concat([base_test, temp_df]).reset_index(drop=True)
    
    return base_test


########################### Helper to make dynamic rolling lags
#################################################################################
def make_lag(LAG_DAY):
    lag_df = base_test[['id','d',TARGET]]
    col_name = 'sales_lag_'+str(LAG_DAY)
    lag_df[col_name] = lag_df.groupby(['id'])[TARGET].transform(lambda x: x.shift(LAG_DAY)).astype(np.float16)
    return lag_df[[col_name]]


def make_lag_roll(LAG_DAY):
    shift_day = LAG_DAY[0]
    roll_wind = LAG_DAY[1]
    lag_df = base_test[['id','d',TARGET]]
    col_name = 'rolling_mean_tmp_'+str(shift_day)+'_'+str(roll_wind)
    lag_df[col_name] = lag_df.groupby(['id'])[TARGET].transform(lambda x: x.shift(shift_day).rolling(roll_wind).mean())
    return lag_df[[col_name]]

In [0]:
########################### Model params
#################################################################################
import lightgbm as lgb
lgb_params = {
                    'boosting_type': 'gbdt',
                    'objective': 'tweedie',
                    'tweedie_variance_power': 1.1,
                    'metric': 'rmse',
                    'subsample': 0.5,
                    'subsample_freq': 1,
                    'learning_rate': 0.03,
                    'num_leaves': 2**11-1,
                    'min_data_in_leaf': 2**12-1,
                    'feature_fraction': 0.5,
                    'max_bin': 100,
                    'n_estimators': 1400,
                    'boost_from_average': False,
                    'verbose': -1,
                } 

# Let's look closer on params

## 'boosting_type': 'gbdt'
# we have 'goss' option for faster training
# but it normally leads to underfit.
# Also there is good 'dart' mode
# but it takes forever to train
# and model performance depends 
# a lot on random factor 
# https://www.kaggle.com/c/home-credit-default-risk/discussion/60921

## 'objective': 'tweedie'
# Tweedie Gradient Boosting for Extremely
# Unbalanced Zero-inflated Data
# https://arxiv.org/pdf/1811.10192.pdf
# and many more articles about tweediie
#
# Strange (for me) but Tweedie is close in results
# to my own ugly loss.
# My advice here - make OWN LOSS function
# https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/140564
# https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/143070
# I think many of you already using it (after poisson kernel appeared) 
# (kagglers are very good with "params" testing and tuning).
# Try to figure out why Tweedie works.
# probably it will show you new features options
# or data transformation (Target transformation?).

## 'tweedie_variance_power': 1.1
# default = 1.5
# set this closer to 2 to shift towards a Gamma distribution
# set this closer to 1 to shift towards a Poisson distribution
# my CV shows 1.1 is optimal 
# but you can make your own choice

## 'metric': 'rmse'
# Doesn't mean anything to us
# as competition metric is different
# and we don't use early stoppings here.
# So rmse serves just for general 
# model performance overview.
# Also we use "fake" validation set
# (as it makes part of the training set)
# so even general rmse score doesn't mean anything))
# https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/133834

## 'subsample': 0.5
# Serves to fight with overfit
# this will randomly select part of data without resampling
# Chosen by CV (my CV can be wrong!)
# Next kernel will be about CV

##'subsample_freq': 1
# frequency for bagging
# default value - seems ok

## 'learning_rate': 0.03
# Chosen by CV
# Smaller - longer training
# but there is an option to stop 
# in "local minimum"
# Bigger - faster training
# but there is a chance to
# not find "global minimum" minimum

## 'num_leaves': 2**11-1
## 'min_data_in_leaf': 2**12-1
# Force model to use more features
# We need it to reduce "recursive"
# error impact.
# Also it leads to overfit
# that's why we use small 
# 'max_bin': 100

## l1, l2 regularizations
# https://towardsdatascience.com/l1-and-l2-regularization-methods-ce25e7fc831c
# Good tiny explanation
# l2 can work with bigger num_leaves
# but my CV doesn't show boost
                    
## 'n_estimators': 1400
# CV shows that there should be
# different values for each state/store.
# Current value was chosen 
# for general purpose.
# As we don't use any early stopings
# careful to not overfit Public LB.

##'feature_fraction': 0.5
# LightGBM will randomly select 
# part of features on each iteration (tree).
# We have maaaany features
# and many of them are "duplicates"
# and many just "noise"
# good values here - 0.5-0.7 (by CV)

## 'boost_from_average': False
# There is some "problem"
# to code boost_from_average for 
# custom loss
# 'True' makes training faster
# BUT carefull use it
# https://github.com/microsoft/LightGBM/issues/1514
# not our case but good to know cons

In [0]:
########################### Vars
#################################################################################
VER = 6                          # Our model version
SEED = 42                        # We want all things
seed_everything(SEED)            # to be as deterministic 
lgb_params['seed'] = SEED        # as possible
N_CORES = psutil.cpu_count()     # Available CPU cores


#LIMITS and const
TARGET      = 'sales'            # Our target
START_TRAIN = 0                  # We can skip some rows (Nans/faster training)
END_TRAIN   = 1913               # End day of our train set
P_HORIZON   = 28                 # Prediction horizon
USE_AUX     = False            # Use or not pretrained models

#FEATURES to remove
## These features lead to overfit
## or values not present in test set
remove_features = ['id','state_id','store_id',
                   'date','wm_yr_wk','d','snap_CA','snap_TX','snap_WI'
                    ,
                   TARGET]
mean_features   = ['enc_cat_id_mean','enc_cat_id_std',
                   'enc_dept_id_mean','enc_dept_id_std',
                   'enc_item_id_mean','enc_item_id_std'] 

#PATHS for Features
ORIGINAL = DIRPATH
BASE     = DIRPATH+'/output/m5-simple-fe/grid_part_1.pkl'
PRICE    = DIRPATH+'/output/m5-simple-fe/grid_part_2.pkl'
CALENDAR = DIRPATH+'/output/m5-simple-fe/grid_part_3.pkl'
LAGS     = DIRPATH+'/output/m5-lags-features/lags_df_28.pkl'
MEAN_ENC = DIRPATH+'/output/m5-custom-features/mean_encoding_df.pkl'
FINAL_SALES = DIRPATH+'/output/m5-custom-features/finalsales_df.pkl'
SNAP      =  DIRPATH+'/output/m5-custom-features/snap_df.pkl'
EVENT_TYPE      =  DIRPATH+'/output/m5-custom-features/event_type.pkl'
EVENT_VALUE      =  DIRPATH+'/output/m5-custom-features/event_value.pkl'
LAGS_364=  DIRPATH+'/output/m5-custom-features/lags_df_364.pkl'
HOLIDAY=DIRPATH+'/output/m5-custom-features/holiday_mean.pkl'
# AUX(pretrained) Models paths
AUX_MODELS = DIRPATH+'/output/m5-three-shades-of-dark-darker-magic/'


#STORES ids
STORES_IDS = pd.read_csv(ORIGINAL+'sales_train_validation.csv')['store_id']
STORES_IDS = list(STORES_IDS.unique())


#SPLITS for lags creation
SHIFT_DAY  = 28
N_LAGS     = 15
LAGS_SPLIT = [col for col in range(SHIFT_DAY,SHIFT_DAY+N_LAGS)]
ROLS_SPLIT = []
for i in [1,7,14]:
    for j in [7,14,30,60]:
        ROLS_SPLIT.append([i,j])

In [0]:
grid_df, features_columns = get_data_by_store("CA_1")


In [0]:
train_mask = grid_df['d']<=END_TRAIN
valid_mask = train_mask&(grid_df['d']>(END_TRAIN-P_HORIZON))
preds_mask = grid_df['d']>(END_TRAIN-100)

In [0]:
#欠損値
#grid_df1=grid_df[~grid_df.isin([np.nan, np.inf, -np.inf]).any(1)]
#grid_df1.head()
#grid_df.loc[:,~grid_df.notnull().all()]
category_col=["sales","id","d","item_id","dept_id","cat_id","event_name_1","event_type_1","event_name_2","event_type_2"]
tmp_col=[x for x in grid_df.columns if x not in category_col]
grid_df.loc[:,tmp_col]=grid_df[tmp_col].fillna(0)
#grid_df=pd.concat([grid_df[category_col],grid_df[tmp_col].fillna(0)],axis=1)

In [14]:
grid_df.head()

,id,d,sales,item_id,dept_id,cat_id,release,sell_price,price_max,price_min,price_std,price_mean,price_norm,price_nunique,item_nunique,price_momentum,price_momentum_m,price_momentum_y,event_name_1,event_type_1,event_name_2,event_type_2,tm_d,tm_w,tm_m,tm_y,tm_wm,tm_dw,tm_w_end,enc_cat_id_mean,enc_cat_id_std,enc_dept_id_mean,enc_dept_id_std,enc_item_id_mean,enc_item_id_std,sales_lag_28,sales_lag_29,sales_lag_30,sales_lag_31,sales_lag_32,...,rolling_mean_14,rolling_std_14,rolling_mean_30,rolling_std_30,rolling_mean_60,rolling_std_60,rolling_mean_180,rolling_std_180,rolling_mean_tmp_1_7,rolling_mean_tmp_1_14,rolling_mean_tmp_1_30,rolling_mean_tmp_1_60,rolling_mean_tmp_7_7,rolling_mean_tmp_7_14,rolling_mean_tmp_7_30,rolling_mean_tmp_7_60,rolling_mean_tmp_14_7,rolling_mean_tmp_14_14,rolling_mean_tmp_14_30,rolling_mean_tmp_14_60,snap,Event_total,Event_total,Event_total,Event_total,sales_lag_28_364,sales_lag_35_364,rolling_mean_7_364,rolling_std_7_364,rolling_mean_14_364,rolling_std_14_364,rolling_mean_30_364,rolling_std_30_364,lag_28_35_mean_364,holiday,rolling_holiday_mean_7,rolling_holiday_mean_14,rolling_holiday_mean_30,rolling_holiday_mean_60,rolling_holiday_mean_180
0,HOBBIES_1_008_CA_1_validation,1,12.0,HOBBIES_1_008,HOBBIES_1,HOBBIES,0,0.459961,0.500000,0.419922,0.019760,0.476318,0.919922,4.0,16,0.0,0.968750,0.949219,NaN,NaN,NaN,NaN,29,4,1,0,5,5,1,0.708984,2.259766,0.865234,2.544922,4.695312,7.183594,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,0.0
1,HOBBIES_1_009_CA_1_validation,1,2.0,HOBBIES_1_009,HOBBIES_1,HOBBIES,0,1.559570,1.769531,1.559570,0.032745,1.764648,0.881348,2.0,9,0.0,0.885742,0.896484,NaN,NaN,NaN,NaN,29,4,1,0,5,5,1,0.708984,2.259766,0.865234,2.544922,0.850098,1.754883,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,0.0
2,HOBBIES_1_010_CA_1_validation,1,0.0,HOBBIES_1_010,HOBBIES_1,HOBBIES,0,3.169922,3.169922,2.970703,0.046356,2.980469,1.000000,2.0,20,0.0,1.064453,1.043945,NaN,NaN,NaN,NaN,29,4,1,0,5,5,1,0.708984,2.259766,0.865234,2.544922,0.611328,0.863281,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,0.0
3,HOBBIES_1_012_CA_1_validation,1,0.0,HOBBIES_1_012,HOBBIES_1,HOBBIES,0,5.980469,6.519531,5.980469,0.115967,6.468750,0.916992,3.0,71,0.0,0.921875,0.958984,NaN,NaN,NaN,NaN,29,4,1,0,5,5,1,0.708984,2.259766,0.865234,2.544922,0.384766,0.692871,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,0.0
4,HOBBIES_1_015_CA_1_validation,1,4.0,HOBBIES_1_015,HOBBIES_1,HOBBIES,0,0.700195,0.720215,0.680176,0.011337,0.706543,0.972168,3.0,16,0.0,0.990234,1.001953,NaN,NaN,NaN,NaN,29,4,1,0,5,5,1,0.708984,2.259766,0.865234,2.544922,4.441406,6.703125,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,0.0


In [0]:


str_col=["event_name_1","event_type_1","event_name_2","event_type_2"]
grid_df[str_col]=grid_df[str_col].astype(str)
tmp_col=[x for x in grid_df.columns if x not in str_col]
grid_df.loc[:,str_col]=grid_df[str_col].fillna("Normal")

#grid_df=pd.concat([grid_df[str_col],grid_df[tmp_col].fillna("Normal")],axis=1)

In [0]:
#Lable Encoder

from sklearn.preprocessing import LabelEncoder
for col in ["item_id","dept_id","cat_id","event_name_1","event_type_1","event_name_2","event_type_2"]:
  #LabelEncoderのインスタンスを生成
  le = LabelEncoder()
  #ラベルを覚えさせる
  le = le.fit(grid_df[col])
  #ラベルを整数に変換
  grid_df[col] = le.transform(grid_df[col])

In [0]:
X_train=grid_df[train_mask][features_columns]
y_train=grid_df[train_mask][TARGET]
X_valid=grid_df[valid_mask][features_columns]
y_valid=grid_df[valid_mask][TARGET]

In [18]:
#X_valid[X_valid.isin([np.nan, np.inf, -np.inf]).any(1)]
X_train.isnull().any()

item_id                     False
dept_id                     False
cat_id                      False
release                     False
sell_price                  False
                            ...  
rolling_holiday_mean_7      False
rolling_holiday_mean_14     False
rolling_holiday_mean_30     False
rolling_holiday_mean_60     False
rolling_holiday_mean_180    False
Length: 93, dtype: bool

In [21]:
#####Borura
# 全部の特徴量で学習
model = lgb.LGBMRegressor(objective='tweedie',
                          boosting_type="gbdt",
                          tweedie_variance_power=1.1,
                          subsample=0.5,
                          subsample_freq=1,
                          min_data_in_leaf=2**12-1,
                          feature_fraction=0.5,
                          max_bin=100,
                          boost_from_average=False,

                                num_leaves =2**11-1,
                                learning_rate=0.03,
                                n_estimators=300,
                                metric="rmse",
                                eval_set=[[X_valid.values, y_valid.values]]
                          )
model.fit(X_train.values, y_train.values)
y_pred=model.predict(X_train)
print('SCORE with ALL Features: %1.2f\n' % np.sqrt(mean_squared_error(y_train,model.predict(X_train))))
print('SCORE with ALL Features: %1.2f\n' % np.sqrt(mean_squared_error(y_valid,model.predict(X_valid))))

SCORE with ALL Features: 2.49

SCORE with ALL Features: 2.01



In [22]:
###ボルタで選択
model = lgb.LGBMRegressor(objective='tweedie',
                          boosting_type="gbdt",
                          tweedie_variance_power=1.1,
                          subsample=0.5,
                          subsample_freq=1,
                          min_data_in_leaf=2**12-1,
                          feature_fraction=0.5,
                          max_bin=100,
                          boost_from_average=False,

                                num_leaves =2**11-1,
                                learning_rate=0.03,
                                n_estimators=300,
                                metric="rmse",
                                eval_set=[[X_valid.values, y_valid.values]]
                          )

feat_selector = BorutaPyForLGB(model, n_estimators='auto', two_step=True,max_iter=500,perc=80,verbose=2, random_state=42)
feat_selector.fit(X_train.values, y_train.values)
print(X_train.columns[feat_selector.support_])

Iteration: 	1 / 500
Confirmed: 	0
Tentative: 	93
Rejected: 	0
Iteration: 	2 / 500
Confirmed: 	0
Tentative: 	93
Rejected: 	0
Iteration: 	3 / 500
Confirmed: 	0
Tentative: 	93
Rejected: 	0
Iteration: 	4 / 500
Confirmed: 	0
Tentative: 	93
Rejected: 	0
Iteration: 	5 / 500
Confirmed: 	0
Tentative: 	93
Rejected: 	0
Iteration: 	6 / 500
Confirmed: 	0
Tentative: 	93
Rejected: 	0
Iteration: 	7 / 500
Confirmed: 	0
Tentative: 	93
Rejected: 	0
Iteration: 	8 / 500
Confirmed: 	61
Tentative: 	5
Rejected: 	27
Iteration: 	9 / 500
Confirmed: 	61
Tentative: 	5
Rejected: 	27
Iteration: 	10 / 500
Confirmed: 	61
Tentative: 	5
Rejected: 	27
Iteration: 	11 / 500
Confirmed: 	61
Tentative: 	5
Rejected: 	27
Iteration: 	12 / 500
Confirmed: 	62
Tentative: 	4
Rejected: 	27
Iteration: 	13 / 500
Confirmed: 	62
Tentative: 	4
Rejected: 	27
Iteration: 	14 / 500
Confirmed: 	62
Tentative: 	4
Rejected: 	27
Iteration: 	15 / 500
Confirmed: 	62
Tentative: 	4
Rejected: 	27
Iteration: 	16 / 500
Confirmed: 	63
Tentative: 	3
Reject

In [23]:
 # 選択したFeatureを取り出し
X_train_selected = X_train.iloc[:,feat_selector.support_]
X_valid_selected = X_valid.iloc[:,feat_selector.support_]
print(X_valid_selected.head())

         item_id  dept_id  ...  rolling_holiday_mean_60  rolling_holiday_mean_180
4617523     1437        3  ...                 0.897461                  0.716797
4617524     1438        3  ...                 0.153809                  0.324951
4617525     1439        3  ...                 0.205078                  0.583496
4617526     1440        3  ...                 1.692383                  1.458008
4617527     1441        3  ...                 0.743652                  1.083008

[5 rows x 65 columns]


In [28]:
feature_df = pd.DataFrame(X_train.columns.tolist(), columns=['features'])
feature_df['rank']=feat_selector.ranking_
feature_df = feature_df.sort_values('rank', ascending=True).reset_index(drop=True)
print ('\n Top %d features:' % feat_selector.n_features_)
print (feature_df.head(feat_selector.n_features_))
feature_df.to_csv('boruta-feature-ranking.csv', index=False)


 Top 65 features:
                 features  rank
0                 item_id     1
1   rolling_mean_tmp_7_30     1
2   rolling_mean_tmp_7_14     1
3    rolling_mean_tmp_7_7     1
4   rolling_mean_tmp_1_60     1
..                    ...   ...
60       enc_dept_id_mean     1
61        enc_dept_id_std     1
62       enc_item_id_mean     1
63        enc_item_id_std     1
64           sales_lag_28     1

[65 rows x 2 columns]


In [29]:
####選択した特徴量で学習
model = lgb.LGBMRegressor(objective='tweedie',
                          boosting_type="gbdt",
                          tweedie_variance_power=1.1,
                          subsample=0.5,
                          subsample_freq=1,
                          min_data_in_leaf=2**12-1,
                          feature_fraction=0.5,
                          max_bin=100,
                          boost_from_average=False,

                                num_leaves =2**11-1,
                                learning_rate=0.03,
                                n_estimators=300,
                                metric="rmse",
                                eval_set=[[X_valid_selected.values, y_valid.values]]
                          )
model.fit(X_train_selected.values, y_train.values)
print('SCORE with ALL Features: %1.2f\n' % np.sqrt(mean_squared_error(y_train,model.predict(X_train_selected))))
print('SCORE with ALL Features: %1.2f\n' % np.sqrt(mean_squared_error(y_valid,model.predict(X_valid_selected))))


SCORE with ALL Features: 2.49

SCORE with ALL Features: 2.02



In [0]:
ボルタ2回目

In [28]:
###ボルタで選択
model = lgb.LGBMRegressorr(objective='tweedie',
                          boosting_type="gbdt",
                          tweedie_variance_power=1.1,
                          subsample=0.5,
                          subsample_freq=1,
                          min_data_in_leaf=2**12-1,
                          feature_fraction=0.5,
                          max_bin=100,
                          boost_from_average=False,

                                num_leaves =2**11-1,
                                learning_rate=0.03,
                                n_estimators=300,
                                metric="rmse",
                                eval_set=[[X_valid_selected.values, y_valid.values]]
                          )

feat_selector2 = BorutaPyForLGB(model, n_estimators='auto', two_step=True,max_iter=500,perc=80,verbose=2, random_state=42)
feat_selector2.fit(X_train_selected.values, y_train.values)
print(X_train.columns[feat_selector.support_])

Iteration: 	1 / 500
Confirmed: 	0
Tentative: 	68
Rejected: 	0
Iteration: 	2 / 500
Confirmed: 	0
Tentative: 	68
Rejected: 	0
Iteration: 	3 / 500
Confirmed: 	0
Tentative: 	68
Rejected: 	0
Iteration: 	4 / 500
Confirmed: 	0
Tentative: 	68
Rejected: 	0
Iteration: 	5 / 500
Confirmed: 	0
Tentative: 	68
Rejected: 	0
Iteration: 	6 / 500
Confirmed: 	0
Tentative: 	68
Rejected: 	0
Iteration: 	7 / 500
Confirmed: 	0
Tentative: 	68
Rejected: 	0
Iteration: 	8 / 500
Confirmed: 	61
Tentative: 	0
Rejected: 	7


BorutaPy finished running.

Iteration: 	9 / 500
Confirmed: 	61
Tentative: 	0
Rejected: 	7
Index(['item_id', 'dept_id', 'release', 'sell_price', 'price_max', 'price_min',
       'price_std', 'price_mean', 'price_nunique', 'item_nunique',
       'price_momentum', 'price_momentum_m', 'price_momentum_y',
       'event_name_1', 'event_type_1', 'tm_d', 'tm_w', 'tm_m', 'tm_y', 'tm_dw',
       'tm_w_end', 'enc_cat_id_mean', 'enc_dept_id_mean', 'enc_dept_id_std',
       'enc_item_id_mean', 'enc_item_id_std

In [29]:
 # 選択したFeatureを取り出し
X_train_selected2 = X_train.iloc[:,feat_selector.support_]
X_valid_selected2 = X_valid.iloc[:,feat_selector.support_]
print(X_valid_selected2.head())

         item_id  dept_id  ...  rolling_holiday_mean_60  rolling_holiday_mean_180
4617523     1437        3  ...                 0.897461                  0.716797
4617524     1438        3  ...                 0.153809                  0.324951
4617525     1439        3  ...                 0.205078                  0.583496
4617526     1440        3  ...                 1.692383                  1.458008
4617527     1441        3  ...                 0.743652                  1.083008

[5 rows x 68 columns]


In [31]:
####選択した特徴量で学習
model = lgb.LGBMRegressor(objective='tweedie',
                          boosting_type="gbdt",
                          tweedie_variance_power=1.1,
                          subsample=0.5,
                          subsample_freq=1,
                          min_data_in_leaf=2**12-1,
                          feature_fraction=0.5,
                          max_bin=100,
                          boost_from_average=False,

                                num_leaves =2**11-1,
                                learning_rate=0.03,
                                n_estimators=1400,
                                metric="rmse",
                                eval_set=[[X_valid_selected2.values, y_valid.values]]
                          )
model.fit(X_train_selected2.values, y_train.values)
print('SCORE with ALL Features: %1.2f\n' % np.sqrt(mean_squared_error(y_train,model.predict(X_train_selected2))))
print('SCORE with ALL Features: %1.2f\n' % np.sqrt(mean_squared_error(y_valid,model.predict(X_valid_selected2))))

SCORE with ALL Features: 2.39

SCORE with ALL Features: 2.01



In [0]:
feat_selector = BorutaPy(model, 
                         n_estimators='auto',  # 特徴量の数に比例して、木の本数を増やす
                         verbose=2, # 0: no output,1: displays iteration number,2: which features have been selected already
                         alpha=0.05, # 有意水準
                         max_iter=50, # 試行回数
                         random_state=1
                        )


In [0]:
# RandomForestRegressorでBorutaを実行
feat_selector.fit(X.values, y.values)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 40 concurrent workers.
[Parallel(n_jobs=-1)]: Done 189 out of 189 | elapsed:  2.7min finished


Iteration: 	1 / 50
Confirmed: 	0
Tentative: 	88
Rejected: 	0


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 40 concurrent workers.
[Parallel(n_jobs=-1)]: Done 189 out of 189 | elapsed:  2.6min finished


Iteration: 	2 / 50
Confirmed: 	0
Tentative: 	88
Rejected: 	0


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 40 concurrent workers.
[Parallel(n_jobs=-1)]: Done 189 out of 189 | elapsed:  2.7min finished


...

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 40 concurrent workers.
[Parallel(n_jobs=-1)]: Done 157 out of 157 | elapsed:  1.8min finished


Iteration: 	48 / 50
Confirmed: 	52
Tentative: 	9
Rejected: 	27


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 40 concurrent workers.
[Parallel(n_jobs=-1)]: Done 157 out of 157 | elapsed:  1.8min finished


Iteration: 	49 / 50
Confirmed: 	52
Tentative: 	9
Rejected: 	27


BorutaPy finished running.

Iteration: 	50 / 50
Confirmed: 	52
Tentative: 	4
Rejected: 	27


BorutaPy(alpha=0.05,
         estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                         criterion='mse', max_depth=7,
                                         max_features='sqrt',
                                         max_leaf_nodes=None, max_samples=None,
                                         min_impurity_decrease=0.0,
                                         min_impurity_split=None,
                                         min_samples_leaf=1,
                                         min_samples_split=2,
                                         min_weight_fraction_leaf=0.0,
                                         n_estimators=157, n_jobs=-1,
                                         oob_score=False,
                                         random_state=RandomState(MT19937) at 0x7F749DF49DB0,
                                         verbose=True, warm_start=False),
         max_iter=50, n_estimators='auto', perc=100,
         random_s

In [0]:
# 選択された特徴量を確認
selected = feat_selector.support_
print('選択された特徴量の数: %d' % np.sum(selected))
print(selected)
print(X.columns[selected])
print(X.columns[~selected])


選択された特徴量の数: 52
[False False False False  True False False False False  True False False
 False  True  True False False False False False  True  True False False
 False False  True  True  True  True  True  True False False  True  True
  True False False False False  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True False False False False False False False
 False False  True  True  True  True  True  True  True  True  True  True
 False  True  True  True]
Index(['sell_price', 'price_norm', 'price_momentum_m', 'price_momentum_y',
       'tm_dw', 'tm_w_end', 'enc_item_id_mean', 'enc_item_id_std',
       'sales_lag_28', 'sales_lag_29', 'sales_lag_30', 'sales_lag_31',
       'sales_lag_34', 'sales_lag_35', 'sales_lag_36', 'sales_lag_41',
       'sales_lag_42', 'rolling_mean_7', 'rolling_std_7', 'rolling_mean_14',
       'rolling_std_14', 'rolling_mean_30', 'rolling_std_30',
       'rolling_mean_60',

In [0]:
# 選択した特徴量で学習
X_selected = X[X.columns[selected]]
rf2 = RandomForestRegressor(
    n_estimators=50
    , criterion='mse'
    , max_depth = 7
    , max_features = 'sqrt' 
    , n_jobs=-1
    , verbose=True
    )
rf2.fit(X_selected, y)
y_pred=rf2.predict(X_selected)

print('RMSE: %1.2f\n' % np.sqrt(mean_squared_error(y, y_pred)))

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 40 concurrent workers.
[Parallel(n_jobs=-1)]: Done  22 out of  50 | elapsed:   21.7s remaining:   27.6s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:   31.5s finished
[Parallel(n_jobs=40)]: Using backend ThreadingBackend with 40 concurrent workers.
[Parallel(n_jobs=40)]: Done  22 out of  50 | elapsed:    0.5s remaining:    0.6s
[Parallel(n_jobs=40)]: Done  50 out of  50 | elapsed:    0.6s finished


RMSE: 2.44



In [0]:
########################### Aux Models
# If you don't want to wait hours and hours
# to have result you can train each store 
# in separate kernel and then just join result.

# If we want to use pretrained models we can 
## skip training 
## (in our case do dummy training
##  to show that we are good with memory
##  and you can safely use this (all kernel) code)
if USE_AUX:
    lgb_params['n_estimators'] = 2
    
# Here is some 'logs' that can compare
#Train CA_1
#[100]	valid_0's rmse: 2.02289
#[200]	valid_0's rmse: 2.0017
#[300]	valid_0's rmse: 1.99239
#[400]	valid_0's rmse: 1.98471
#[500]	valid_0's rmse: 1.97923
#[600]	valid_0's rmse: 1.97284
#[700]	valid_0's rmse: 1.96763
#[800]	valid_0's rmse: 1.9624
#[900]	valid_0's rmse: 1.95673
#[1000]	valid_0's rmse: 1.95201
#[1100]	valid_0's rmse: 1.9476
#[1200]	valid_0's rmse: 1.9434
#[1300]	valid_0's rmse: 1.9392
#[1400]	valid_0's rmse: 1.93446

#Train CA_2
#[100]	valid_0's rmse: 1.88949
#[200]	valid_0's rmse: 1.84767
#[300]	valid_0's rmse: 1.83653
#[400]	valid_0's rmse: 1.82909
#[500]	valid_0's rmse: 1.82265
#[600]	valid_0's rmse: 1.81725
#[700]	valid_0's rmse: 1.81252
#[800]	valid_0's rmse: 1.80736
#[900]	valid_0's rmse: 1.80242
#[1000]	valid_0's rmse: 1.79821
#[1100]	valid_0's rmse: 1.794
#[1200]	valid_0's rmse: 1.78973
#[1300]	valid_0's rmse: 1.78552
#[1400]	valid_0's rmse: 1.78158

In [0]:
########################### Train Models
#################################################################################
for store_id in STORES_IDS:
    print('Train', store_id)
    
    # Get grid for current store
    grid_df, features_columns = get_data_by_store(store_id)
    
    # Masks for 
    # Train (All data less than 1913)
    # "Validation" (Last 28 days - not real validatio set)
    # Test (All data greater than 1913 day, 
    #       with some gap for recursive features)
    train_mask = grid_df['d']<=END_TRAIN
    valid_mask = train_mask&(grid_df['d']>(END_TRAIN-P_HORIZON))
    preds_mask = grid_df['d']>(END_TRAIN-100)
    
    # Apply masks and save lgb dataset as bin
    # to reduce memory spikes during dtype convertations
    # https://github.com/Microsoft/LightGBM/issues/1032
    # "To avoid any conversions, you should always use np.float32"
    # or save to bin before start training
    # https://www.kaggle.com/c/talkingdata-adtracking-fraud-detection/discussion/53773
    train_data = lgb.Dataset(grid_df[train_mask][features_columns], 
                       label=grid_df[train_mask][TARGET])
    train_data.save_binary('train_data.bin')
    train_data = lgb.Dataset('train_data.bin')
    
    valid_data = lgb.Dataset(grid_df[valid_mask][features_columns], 
                       label=grid_df[valid_mask][TARGET])
    
    # Saving part of the dataset for later predictions
    # Removing features that we need to calculate recursively 
    grid_df = grid_df[preds_mask].reset_index(drop=True)
    keep_cols = [col for col in list(grid_df) if '_tmp_' not in col]
    grid_df = grid_df[keep_cols]
    grid_df.to_pickle('test_'+store_id+'.pkl')
    del grid_df
    
    # Launch seeder again to make lgb training 100% deterministic
    # with each "code line" np.random "evolves" 
    # so we need (may want) to "reset" it
    seed_everything(SEED)
    estimator = lgb.train(lgb_params,
                          train_data,
                          valid_sets = [valid_data],
                          verbose_eval = 100,
                          )
    
    # Save model - it's not real '.bin' but a pickle file
    # estimator = lgb.Booster(model_file='model.txt')
    # can only predict with the best iteration (or the saving iteration)
    # pickle.dump gives us more flexibility
    # like estimator.predict(TEST, num_iteration=100)
    # num_iteration - number of iteration want to predict with, 
    # NULL or <= 0 means use best iteration
    model_name = AUX_MODELS+'lgb_model_'+store_id+'_v'+str(VER)+'.bin'
    pickle.dump(estimator, open(model_name, 'wb'))

    # Remove temporary files and objects 
    # to free some hdd space and ram memory
    !rm train_data.bin
    del train_data, valid_data, estimator
    gc.collect()
    
    # "Keep" models features for predictions
    MODEL_FEATURES = features_columns

Train CA_1


In [0]:
########################### Predict
#################################################################################

# Create Dummy DataFrame to store predictions
all_preds = pd.DataFrame()

# Join back the Test dataset with 
# a small part of the training data 
# to make recursive features
base_test = get_base_test()

# Timer to measure predictions time 
main_time = time.time()

# Loop over each prediction day
# As rolling lags are the most timeconsuming
# we will calculate it for whole day
for PREDICT_DAY in range(1,29):    
    print('Predict | Day:', PREDICT_DAY)
    start_time = time.time()

    # Make temporary grid to calculate rolling lags
    grid_df = base_test.copy()
    grid_df = pd.concat([grid_df, df_parallelize_run(make_lag_roll, ROLS_SPLIT)], axis=1)
        
    for store_id in STORES_IDS:
        
        # Read all our models and make predictions
        # for each day/store pairs
        model_path = AUX_MODELS + 'lgb_model_'+store_id+'_v'+str(VER)+'.bin' 
        if USE_AUX:
            model_path = AUX_MODELS + model_path
        
        estimator = pickle.load(open(model_path, 'rb'))
        
        day_mask = base_test['d']==(END_TRAIN+PREDICT_DAY)
        store_mask = base_test['store_id']==store_id
        
        mask = (day_mask)&(store_mask)
        base_test[TARGET][mask] = estimator.predict(grid_df[mask][MODEL_FEATURES])
    
    # Make good column naming and add 
    # to all_preds DataFrame
    temp_df = base_test[day_mask][['id',TARGET]]
    temp_df.columns = ['id','F'+str(PREDICT_DAY)]
    if 'id' in list(all_preds):
        all_preds = all_preds.merge(temp_df, on=['id'], how='left')
    else:
        all_preds = temp_df.copy()
        
    print('#'*10, ' %0.2f min round |' % ((time.time() - start_time) / 60),
                  ' %0.2f min total |' % ((time.time() - main_time) / 60),
                  ' %0.2f day sales |' % (temp_df['F'+str(PREDICT_DAY)].sum()))
    del temp_df
    
all_preds = all_preds.reset_index(drop=True)
all_preds

Predict | Day: 1
##########  0.89 min round |  0.89 min total |  37443.86 day sales |
Predict | Day: 2
##########  0.65 min round |  1.54 min total |  35370.47 day sales |
Predict | Day: 3
##########  0.64 min round |  2.18 min total |  34752.00 day sales |
Predict | Day: 4
##########  0.64 min round |  2.82 min total |  35318.04 day sales |
Predict | Day: 5
##########  0.64 min round |  3.46 min total |  41597.11 day sales |
Predict | Day: 6
##########  0.64 min round |  4.10 min total |  52161.03 day sales |
Predict | Day: 7
##########  0.64 min round |  4.74 min total |  54737.23 day sales |
Predict | Day: 8
##########  0.64 min round |  5.38 min total |  44382.08 day sales |
Predict | Day: 9
##########  0.64 min round |  6.02 min total |  44672.14 day sales |
Predict | Day: 10
##########  0.65 min round |  6.67 min total |  39121.28 day sales |
Predict | Day: 11
##########  0.64 min round |  7.31 min total |  39305.85 day sales |
Predict | Day: 12
##########  0.65 min round |  7.96

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0.801803,0.723683,0.720562,0.762624,0.960074,1.126830,1.161501,0.846897,0.911767,0.834176,0.746047,0.949804,1.120835,0.938112,0.856383,0.791389,0.787631,0.735187,0.810740,1.098141,1.063745,0.814246,0.782200,0.739718,0.801988,0.955650,1.096880,1.022920
1,HOBBIES_1_002_CA_1_validation,0.168957,0.169508,0.166854,0.179415,0.200086,0.248333,0.286910,0.230361,0.220693,0.194009,0.184153,0.224189,0.271640,0.249853,0.270110,0.243126,0.224538,0.211191,0.222313,0.302337,0.319417,0.232647,0.209414,0.195903,0.208283,0.211919,0.293963,0.300624
2,HOBBIES_1_003_CA_1_validation,0.421205,0.385268,0.428610,0.409224,0.588067,0.772068,0.719824,0.498321,0.461623,0.456616,0.416941,0.606072,0.686885,0.526176,0.447994,0.435795,0.412512,0.480216,0.567431,0.688343,0.674192,0.479738,0.427863,0.446494,0.436994,0.552482,0.683273,0.664283
3,HOBBIES_1_004_CA_1_validation,1.569804,1.263994,1.293363,1.424997,1.819924,3.211554,3.341809,1.685654,1.450558,1.445097,1.505973,1.817285,3.099209,2.638359,1.672946,1.383769,1.409273,1.348868,1.839778,2.596627,3.379442,1.649361,1.455321,1.357194,1.374343,1.871913,3.126521,3.394666
4,HOBBIES_1_005_CA_1_validation,0.956261,0.833377,0.859554,0.887796,1.001389,1.360069,1.331363,0.975193,1.002860,0.946965,0.819512,0.979274,1.361654,1.032811,0.904866,0.950282,0.865686,0.906523,1.005970,1.467324,1.546693,1.020751,0.894866,0.912960,0.862583,1.111127,1.470302,1.419945
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_validation,0.324340,0.302619,0.304654,0.297616,0.334883,0.478178,0.441987,0.422721,0.469880,0.346602,0.415468,0.426395,0.478824,0.519497,0.440944,0.374909,0.422686,0.406410,0.390496,0.494215,0.571374,0.417284,0.422543,0.396604,0.352524,0.347221,0.410777,0.465673
30486,FOODS_3_824_WI_3_validation,0.304704,0.262193,0.262746,0.254403,0.292248,0.378496,0.405401,0.439849,0.449078,0.363105,0.384419,0.411730,0.453814,0.409679,0.421899,0.312726,0.363811,0.360991,0.319476,0.470812,0.483289,0.356131,0.284941,0.261170,0.243323,0.264301,0.346882,0.323683
30487,FOODS_3_825_WI_3_validation,0.685554,0.552145,0.482045,0.518565,0.658581,0.745489,0.930951,1.174510,1.135844,0.758195,1.034439,1.241368,1.247778,1.354342,1.434308,0.852096,1.153077,1.179667,0.957669,1.439983,1.500104,1.036766,0.810120,0.686931,0.595563,0.735723,0.884382,0.914370
30488,FOODS_3_826_WI_3_validation,0.912796,0.837028,0.728609,0.753934,0.909931,1.228229,1.150880,1.111630,1.135888,0.889787,1.015968,1.197305,1.223163,1.317606,1.197976,0.991891,1.035252,1.084933,0.982257,1.327508,1.409575,0.969228,0.881856,0.859823,0.805324,0.932125,1.073434,1.131151


In [0]:
########################### Export
#################################################################################
# Reading competition sample submission and
# merging our predictions
# As we have predictions only for "_validation" data
# we need to do fillna() for "_evaluation" items
submission = pd.read_csv(ORIGINAL+'sample_submission.csv')[['id']]
submission = submission.merge(all_preds, on=['id'], how='left').fillna(0)
submission.to_csv(AUX_MODELS+'submission_v'+str(VER)+'.csv', index=False)
submission.to_csv('submission_v'+str(VER)+'.csv', index=False)



In [0]:
# Summary

# Of course here is no magic at all.
# No "Novel" features and no brilliant ideas.
# We just carefully joined all
# our previous fe work and created a model.

# Also!
# In my opinion this strategy is a "dead end".
# Overfits a lot LB and with 1 final submission 
# you have no option to risk.


# Improvement should come from:
# Loss function
# Data representation
# Stable CV
# Good features reduction strategy
# Predictions stabilization with NN
# Trend prediction
# Real zero sales detection/classification


# Good kernels references 
## (the order is random and the list is not complete):
# https://www.kaggle.com/ragnar123/simple-lgbm-groupkfold-cv
# https://www.kaggle.com/jpmiller/grouping-items-by-stockout-pattern
# https://www.kaggle.com/headsortails/back-to-predict-the-future-interactive-m5-eda
# https://www.kaggle.com/sibmike/m5-out-of-stock-feature
# https://www.kaggle.com/mayer79/m5-forecast-attack-of-the-data-table
# https://www.kaggle.com/yassinealouini/seq2seq
# https://www.kaggle.com/kailex/m5-forecaster-v2
# https://www.kaggle.com/aerdem4/m5-lofo-importance-on-gpu-via-rapids-xgboost


# Features were created in these kernels:
## 
# Mean encodings and PCA options
# https://www.kaggle.com/kyakovlev/m5-custom-features
##
# Lags and rolling lags
# https://www.kaggle.com/kyakovlev/m5-lags-features
##
# Base Grid and base features (calendar/price/etc)
# https://www.kaggle.com/kyakovlev/m5-simple-fe


# Personal request
# Please don't upvote any ensemble and copypaste kernels
## The worst case is ensemble without any analyse.
## The best choice - just ignore it.
## I would like to see more kernels with interesting and original approaches.
## Don't feed copypasters with upvotes.

## It doesn't mean that you should not fork and improve others kernels
## but I would like to see params and code tuning based on some CV and analyse
## and not only on LB probing.
## Small changes could be shared in comments and authors can improve their kernel.

## Feel free to criticize this kernel as my knowlege is very limited
## and I can be wrong in code and descriptions. 
## Thank you.

In [0]:
grid_df, features_columns = get_data_by_store("CA_1")

In [0]:
grid_df.columns

In [0]:
features_columns

In [0]:
imp=pd.DataFrame({'feature': features_columns,

 'importance':estimator.feature_importance()}).sort_values('importance',

 ascending=False)
imp.to_csv(AUX_MODELS+'importance_v'+str(VER)+'.csv', index=False)